# notes

https://blog.jupyter.org/interactive-gis-in-jupyter-with-ipyleaflet-52f9657fa7a

- voila
- normalization?
- documentation
- functions in modules

# import packages

In [4]:
from ipyleaflet import (Map, GeoData, basemaps, WidgetControl, GeoJSON, LayersControl,
                        Choropleth, Heatmap, SearchControl,FullScreenControl, LegendControl)
from ipywidgets import Text, HTML, widgets, interact, Label, Layout, HBox, VBox
from branca.colormap import linear

import geopandas as gpd
import pandas as pd
pd.options.plotting.backend = "plotly"

import json
import numpy as np
import plotly.express as px
import plotly.graph_objs as go

from datenguidepy import get_regions, get_statistics, Query

# inspect datenguide API

In [5]:
get_regions().query("level == 'nuts1'")

,name,level,parent
region_id,,,
10,Saarland,nuts1,DG
11,Berlin,nuts1,DG
12,Brandenburg,nuts1,DG
13,Mecklenburg-Vorpommern,nuts1,DG
14,Sachsen,nuts1,DG
15,Sachsen-Anhalt,nuts1,DG
16,Thüringen,nuts1,DG
01,Schleswig-Holstein,nuts1,DG
02,Hamburg,nuts1,DG


**=> Hesse has region code '06'**

# Define functions to load datenguide data into data frames

In [6]:
# default values (for testing)
selected_stats = 'AI1903'
year = 2013

In [7]:
def get_data_all_years(selected_stats=selected_stats):
    '''
    For a given `selected_stats` returns data and unit, where
    data [dataframe]: the chosen statistics from datenguide.py with columns 'name', 'year', 'id' and stat values
    units [string]: name of corresponding unit
    '''
    q = Query.all_regions(parent='06')
    stat = q.add_field(selected_stats)
    description = stat.description()

    data = q.results(verbose_enums=True, add_units = True)
    # for some reason entries are produced twice; remove them
    data.drop_duplicates(inplace=True)
    data, unit = data[['name', 'year', 'id', selected_stats]], data[selected_stats+'_unit'].iloc[0]

    return data, unit

def get_statistics_description(selected_stats=selected_stats):
    '''
    get the description string for statistics 'selected_stats'
    '''
    q = Query.all_regions(parent='06')
    stat = q.add_field(selected_stats)
    return stat.description()

# load geoJSON data

the geoJSON file was obtained from http://opendatalab.de/projects/geojson-utilities/  
(contains only data for Hessen on level NUTS3)

In [8]:
geojson_data = json.load(open('../data/landkreise_simplify200.geojson','r'))

In [9]:
# set property 'name' for the county name to make it consistent with datenguide data
for feature in geojson_data['features']:
    feature['properties']['name'] = feature['properties']['BEZ']

write to `geo_data` with identifier key named as `id`  
TODO: why introduce a new variable here?

In [10]:
geo_data = {'features':[], 'id':[]}
for f in geojson_data['features']:
    f.update(id=f['properties']['AGS']) #f['properties']['GEN'])
    geo_data['features'].append(f)

# define and load all statistics that will be available in the app

In [12]:
stat_ids = ['AI1903', 'AI1904', 'FLC048']
stat_descriptions = [get_statistics_description(si) for si in stat_ids]
stat_tuple = tuple(zip(stat_descriptions, stat_ids))
stat_dict = dict((y, x) for x, y in stat_tuple)

In [13]:
choro_data_complete, units = dict(), dict()
for st in stat_ids:
    choro_data_complete[st], units[st] = get_data_all_years(st)
    
c = choro_data_complete[stat_ids[0]] # take one statistics data set for generating name/id mapping
id_to_name = {ids: c.loc[c['id']==ids, 'name'][0] for ids in c['id'].unique()}

# Building the interface

## preparations

define dropdown widgets for stats and year

In [14]:
stats_widget = widgets.Dropdown(
    options=stat_tuple,
    value= stat_ids[0],
    description='statistics:',
    disabled=False,
)

year_widget = widgets.Dropdown(
    options= range(2010,2019),
    value=2018,
    description='year:',
    disabled=False,
)

vb = VBox([stats_widget, year_widget])

create initial bar chart figure with random values (later overwritten when map is created)

In [35]:
data = [go.Bar(x= np.arange(2010,2020), y=[0]*10)]
layout = {'xaxis':{'title':'year'}, 'title':{'text': 'Frankfurt'}}
fig = go.FigureWidget([data[0],data[0]], layout)
fig.update_layout(legend=dict(
    orientation='h',
    yanchor="bottom",
    y=1.,
    xanchor="center",
    x=0.5
));

## functions for callbacks and interactions

set parameters

In [36]:
cm = linear.Blues_03 # color map for county area coloring

Updating the bar charts on clicking a ditrict. Basic logic:
- `fig.data` contains 2 entries, one for each county to be shown (1st is more recently clicked)
- after clicking, new stats is passed via arguments `district_id, stats_name, stats_data_frame, current_year`
- assign: new stats instead of oldest data; most recent data take place of previously old data **needs refinement!**
- bar layout properties are updated accordingly

In [47]:
def update_figure(district_id, stats_name, stats_data_frame, current_year):
    # add new stattistics
    stats_to_add = stats_data_frame[stats_data_frame['id']==district_id].copy(deep=True)
    stats_to_add['colors'] = 'rgb(129,105,75)'
    stats_to_add.loc[stats_to_add['year']==current_year, 'colors'] = 'gray'
    
    bar = fig.data[1]
    bar.x = stats_to_add['year']
    bar.y = stats_to_add[stats_name]
    bar.marker.color= stats_to_add['colors']
    bar.opacity= 1.
    
    fig.data = (bar,fig.data[0])
    
    fig.data[1].opacity=.5
    fig.data[0].name= id_to_name[district_id]

    fig.update_layout(title=f"{fig.data[0].name} vs. {fig.data[1].name}", template='plotly_white')

In [63]:
def show_map(selected_stats, year):
    html = HTML('''district''')
    html.layout.margin = '-5px 5px 5px 5px'    
    control = WidgetControl(widget=html, position='topright', max_width=230)

    # load selected stats into choro_data_all
    choro_data_all, unit = choro_data_complete[selected_stats], units[selected_stats]
    # for geo plot extract chosen year and assign to choro_data
    choro_data = choro_data_all[choro_data_all['year']==year]
    choro_data = dict(choro_data.drop(columns=['year', 'name']).to_dict('split')['data'])
    
    # initialize bar chart with Frankfurt vs Offenbach
    update_figure('06412', selected_stats, choro_data_all, year)
    update_figure('06413', selected_stats, choro_data_all, year)
    
    # set y-axis label
    fig.update_layout(yaxis_title=f'{stat_dict[selected_stats]} [{unit}]', yaxis={'range':[0,max(choro_data_all[selected_stats])]})
    
    # define chropleth layer for basic geo plotting
    layer = Choropleth(geo_data=geo_data,choro_data=choro_data,colormap=cm,
                       style={'fillOpacity': 0.65, 'dashArray': '10, 10', 'weight':1})
    
    # define GeoJSON layer for click and hover event interactions
    geo_json = GeoJSON(data=geo_data,
                       style={'opacity': 0, 'dashArray': '9', 'fillOpacity': .0, 'weight': 1},
                       hover_style={'color': 'green', 'dashArray': '0', 'fillOpacity': 0.7})
    
    # this is used for update of HTML 
    for f in geo_data['features']:
        f['value'] = choro_data[f['id']] # choro_data[f['properties']['GEN']]

    
    # on hover, the html text filed is updated to show properties of the hovered county
    def update_html(feature,  **kwargs):
        html.value = '''
            <h3>{}</h3><center><b>{}</b>: {} {}</center>
        '''.format(stat_dict[selected_stats],
                   id_to_name[feature['id']],
                   feature['value'],
                   unit)
    
    # this function is called upon a click events and triggers figure update with the arguments passed from the map
    def update_fig_on_click(feature, **kwags):
        update_figure(feature['id'], selected_stats, choro_data_all, year)
    geo_json.on_hover(update_html)
    geo_json.on_click(update_fig_on_click)

    # add layers and controls; set layout parameters
    m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=(50.5,9), zoom=8)
    m.add_layer(layer)
    m.add_layer(geo_json)
    m.add_control(control)
    m.layout.width = '40%'
    m.layout.height = '700px'

    # custom made legend using min/max normalization
    min_value, max_value = min(choro_data.values()), max(choro_data.values())
    legend = LegendControl(
          {str(min_value)+' '+str(unit): cm(0),
          str(min_value+0.5*(max_value-min_value))+' '+str(unit): cm(.5),
          str(max_value)+' '+str(unit): cm(1)},
          name= f"{stat_dict[selected_stats]} ({year})", position="bottomleft")
    m.add_control(legend)
    return HBox([m, fig])#m

In [64]:
interact(show_map, selected_stats=stats_widget, year=year_widget);

interactive(children=(Dropdown(description='statistics:', options=(('Getrennt erfasste Wertstoffe', 'AI1903'),…

# install extensions...

In [53]:
#! jupyter labextension install jupyterlab-plotly@4.9.0
#! jupyter labextension install @jupyter-widgets/jupyterlab-manager plotlywidget@4.9.0
#! jupyter labextension install @jupyter-voila/jupyterlab-preview